## Importing Libraries

In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

## Data Preprocessing

In [3]:
#reading the csv
retail_csv = pd.read_csv("data/Online Retail Data Set.csv", encoding='unicode_escape')
retail_csv.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [10]:
#checking the data type, shape and overall description of the dataset 
retail_csv.info(), retail_csv.shape, retail_csv.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


(None,
 (541909, 8),
        InvoiceNo StockCode                         Description       Quantity  \
 count     541909    541909                              540455  541909.000000   
 unique     25900      4070                                4223            NaN   
 top       573585    85123A  WHITE HANGING HEART T-LIGHT HOLDER            NaN   
 freq        1114      2313                                2369            NaN   
 mean         NaN       NaN                                 NaN       9.552250   
 std          NaN       NaN                                 NaN     218.081158   
 min          NaN       NaN                                 NaN  -80995.000000   
 25%          NaN       NaN                                 NaN       1.000000   
 50%          NaN       NaN                                 NaN       3.000000   
 75%          NaN       NaN                                 NaN      10.000000   
 max          NaN       NaN                                 NaN   80995.00000

In [9]:
#checking missing value per column
retail_csv.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

So, we can notice that:
- there are missing values in `Description` and `CustomerID` 
- `InvoiceDate` needs to be converted to date-time object
- `UnitPrice` and `Quantity` have negative values
- `CustomerID` can be a categorial variable

In [12]:
#Handling the missing values first by droping rows where Description or CustomerID is missing
retail_csv = retail_csv.dropna(subset=['Description','CustomerID'])
retail_csv.isnull().sum(), retail_csv.shape

(InvoiceNo      0
 StockCode      0
 Description    0
 Quantity       0
 InvoiceDate    0
 UnitPrice      0
 CustomerID     0
 Country        0
 dtype: int64,
 (406829, 8))

In [15]:
#Convert InvoiceDate column to date-time object
retail_csv['InvoiceDate'] = pd.to_datetime(retail_csv['InvoiceDate'], dayfirst='True')
retail_csv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [16]:
#Dealing with the negative value from UnitPrice and Quantity
retail_csv = retail_csv[(retail_csv['Quantity'] > 0) & (retail_csv['UnitPrice'] > 0)]
retail_csv.describe(include="all")

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,397884,397884,397884,397884.000000,397884,397884.000000,397884.000000,397884
unique,18532,3665,3877,NaN,NaN,NaN,NaN,37
top,576339,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,NaN,NaN,NaN,United Kingdom
freq,542,2035,2028,NaN,NaN,NaN,NaN,354321
mean,NaN,NaN,NaN,12.988238,2011-07-10 23:41:23.511023360,3.116488,15294.423453,NaN
min,NaN,NaN,NaN,1.000000,2010-12-01 08:26:00,0.001000,12346.000000,NaN
25%,NaN,NaN,NaN,2.000000,2011-04-07 11:12:00,1.250000,13969.000000,NaN
50%,NaN,NaN,NaN,6.000000,2011-07-31 14:39:00,1.950000,15159.000000,NaN
75%,NaN,NaN,NaN,12.000000,2011-10-20 14:33:00,3.750000,16795.000000,NaN
max,NaN,NaN,NaN,80995.000000,2011-12-09 12:50:00,8142.750000,18287.000000,NaN


In [19]:
retail_csv['InvoiceNo'].value_counts(), retail_csv['StockCode'].value_counts(), retail_csv['CustomerID'].value_counts(), retail_csv['Country'].value_counts()

(InvoiceNo
 576339    542
 579196    533
 580727    529
 578270    442
 573576    435
          ... 
 556918      1
 556885      1
 544170      1
 556842      1
 575830      1
 Name: count, Length: 18532, dtype: int64,
 StockCode
 85123A    2035
 22423     1723
 85099B    1618
 84879     1408
 47566     1396
           ... 
 90168        1
 90169        1
 90202A       1
 90118        1
 23843        1
 Name: count, Length: 3665, dtype: int64,
 CustomerID
 17841.0    7847
 14911.0    5675
 14096.0    5111
 12748.0    4595
 14606.0    2700
            ... 
 18184.0       1
 13185.0       1
 16073.0       1
 17948.0       1
 17846.0       1
 Name: count, Length: 4338, dtype: int64,
 Country
 United Kingdom          354321
 Germany                   9040
 France                    8341
 EIRE                      7236
 Spain                     2484
 Netherlands               2359
 Belgium                   2031
 Switzerland               1841
 Portugal                  1462
 Australia    

Now, we will encode `InvoiceNo`, `StockCode`, `CustomerID`, `Country` as categorial variable

In [21]:
#Encoding the categorial variables
categorical_columns = ['InvoiceNo', 'StockCode', 'CustomerID', 'Country']
label_encoders = {col:LabelEncoder() for col in categorical_columns}

for col in categorical_columns:
    retail_csv[col] = label_encoders[col].fit_transform(retail_csv[col])
    
#Feature scaling for UnitPrice and Quantity
scaler = StandardScaler()
retail_csv[['Quantity','UnitPrice']] = scaler.fit_transform(retail_csv[['Quantity','UnitPrice']])

#check the new data
retail_csv.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,3233,WHITE HANGING HEART T-LIGHT HOLDER,-0.038968,2010-12-01 08:26:00,-0.025635,4016,35
1,0,2643,WHITE METAL LANTERN,-0.038968,2010-12-01 08:26:00,0.012377,4016,35
2,0,2847,CREAM CUPID HEARTS COAT HANGER,-0.027816,2010-12-01 08:26:00,-0.016585,4016,35
3,0,2795,KNITTED UNION FLAG HOT WATER BOTTLE,-0.038968,2010-12-01 08:26:00,0.012377,4016,35
4,0,2794,RED WOOLLY HOTTIE WHITE HEART.,-0.038968,2010-12-01 08:26:00,0.012377,4016,35
5,0,1601,SET 7 BABUSHKA NESTING BOXES,-0.061273,2010-12-01 08:26:00,0.205156,4016,35
6,0,751,GLASS STAR FROSTED T-LIGHT HOLDER,-0.038968,2010-12-01 08:26:00,0.051295,4016,35
7,1,1486,HAND WARMER UNION JACK,-0.038968,2010-12-01 08:28:00,-0.057313,4016,35
8,1,1485,HAND WARMER RED POLKA DOT,-0.038968,2010-12-01 08:28:00,-0.057313,4016,35
9,2,3059,ASSORTED COLOUR BIRD ORNAMENT,0.106015,2010-12-01 08:34:00,-0.064553,534,35


## Exploratory Data Analysis

## Building Model

## Training and Evaluation

## Result Visualization